In [1]:
!pip install pytest

In [86]:
class PRF():
    def __init__(self, arity, func):
        self.arity = arity
        self.func = func

    def __call__(self, *args):
        assert len(args) == self.arity, "arity mismatch: arity of the function is {}, but {} arguments are given".format(self.arity, len(args))
        ret = self.func(*args)
        return ret
    
    def __matmul__(self, hs):
        "defines a function by composition"
        assert all(map(lambda x: type(x) is PRF, hs))
        assert self.arity == len(hs)
        
    def __truediv__(self, h):
        "defines a function by recursion, where self is called when x == 0 and h when x > 0"
        assert type(h) is PRF
        assert self.arity + 2 == h.arity
        def f(x, *y):
            if x == 0:
                return self(*y)
            else:
                return h(x-1, f(x-1, *y), *y)
        return PRF(self.arity + 1, f)

In [87]:
Z = PRF(0, lambda: 0)
S = PRF(1, lambda x: x + 1)

In [44]:
Z()

0

In [50]:
Z(1)

AssertionError: arity mismatch: arity of the function is 0, but 1 arguments are given

In [51]:
S()

AssertionError: arity mismatch: arity of the function is 1, but 0 arguments are given

In [52]:
S(3)

4

In [56]:
type(S) is PRF

True

In [57]:
type(Z) is PRF

True

In [72]:
def P(n, i):
    return PRF(n, lambda *xs: xs[i-1])

In [74]:
P(3,2)(1,3,5)

3

In [75]:
P(3,2)(1,3,5,7)

AssertionError: arity mismatch: arity of the function is 3, but 4 arguments are given

In [81]:
add = P(1,1) / P(3,2)
print(add)

In [82]:
add(5,3)

3

In [89]:
PRF(2, lambda x, y: x + y) @ [S, S]

In [8]:
from numpy.random import randint

In [9]:
r = randint(10, size=(5,2))

In [10]:
r

array([[2, 0],
       [6, 9],
       [8, 2],
       [5, 4],
       [2, 8]])

In [21]:
from itertools import starmap
def prodfunc(f,g):
    return lambda x: (f(x), g(x))
computed = list(starmap(prodfunc(S, lambda x: x+1), randint(100, size=(5,1))))
computed

[(7, 7), (20, 20), (15, 15), (40, 40), (20, 20)]

In [26]:
compared = list(starmap(lambda x,y: x == y, computed))
compared

[True, True, True, True, True]

In [27]:
all(compared)

True

In [38]:
def testPRF(f, expected_func, N=5):
    computed = list(starmap(prodfunc(S, expected_func), randint(100, size=(N,f.arity))))
    print(computed)
    compared = list(starmap(lambda x,y: x == y, computed))
    print(compared)
    return all(compared)

In [40]:
testPRF(S, lambda x: x+1)

[(8, 8), (7, 7), (60, 60), (59, 59), (44, 44)]
[True, True, True, True, True]


True

In [41]:
testPRF(Z, lambda: 0)

TypeError: <lambda>() missing 1 required positional argument: 'x'

In [90]:
!pip install pydash

     |████████████████████████████████| 92kB 1.3MB/s eta 0:00:01


In [94]:
import pydash

In [95]:
pydash.arrays.tail([1,2,3,4])

[2, 3, 4]

In [96]:
pydash.arrays.tail([])

[]

In [108]:
import pandas as pd
from pandas import Series, DataFrame
from numpy.random import rand

In [104]:
s = Series([2,4,6,8])
s

0    2
1    4
2    6
3    8
dtype: int64

In [105]:
s.empty

False

In [114]:
df1 = DataFrame(rand(2,3), columns=iter('abc'))
df1

,a,b,c
0,0.389044,0.887683,0.254495
1,0.180228,0.121058,0.037890


In [115]:
df2 = DataFrame(rand(2,3), columns=iter('abc'))
df2

,a,b,c
0,0.793140,0.022203,0.852974
1,0.200941,0.632339,0.861187


In [116]:
df1.append(df2)

,a,b,c
0,0.389044,0.887683,0.254495
1,0.180228,0.121058,0.037890
0,0.793140,0.022203,0.852974
1,0.200941,0.632339,0.861187


In [117]:
df1.append(df2, ignore_index=True)

,a,b,c
0,0.389044,0.887683,0.254495
1,0.180228,0.121058,0.037890
2,0.793140,0.022203,0.852974
3,0.200941,0.632339,0.861187
